In [ ]:
import pandas
import itertools as it
import string
from collections import Counter
import json
import re
import numpy
import nltk

In [ ]:
with open('data/stopwords.txt') as fh:
    stopwords = set(fh.read().splitlines())

In [ ]:
sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
def normalise(text):
    
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ", text)
    text = text.replace("/", " / ")
    text = re.sub(r"@\w+", " <user> ", text)
    text = re.sub(r"[8:=;]['`\-][)d]+|[)d]+['`\-][8:=;]", " <smile> ", text)
    text = re.sub(r"[8:=;]['`\-]p+", " <lolface> ", text)
    text = re.sub(r"[8:=;]['`\-]\(+|\)+['`\-][8:=;]", " <sadface> ", text)
    text = re.sub(r"[8:=;]['`\-][\/|l*]", " <neutralface> ", text)
    text = text.replace(r"<3", " <heart> ")
    text = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ", text)
    text = text.replace(r"#", " <hashtag> ")
    text = re.sub(r"([!?.,()])+", r" \1 ", text)
    
    return text

In [ ]:
def tokenise(text):
    return [t.strip() for t in text.strip().split()]

In [ ]:
def sentence_splitting(text):
    return sentence_detector.tokenize(text)

In [ ]:
def parse_treatment_definitons(definion_file):
    treatment_set = set()
    treatment_mapping = {}
    max_length = 1
    for line in definion_file:
        line = line.strip()
        treatments = line.split(',')
        name = treatments[0]
        for treatment in treatments:
            treatment = tuple(tokenise(normalise(treatment)))
            max_length = max(len(treatment), max_length)
            treatment_set.add(treatment)
            treatment_mapping[treatment] = name
    return treatment_set, treatment_mapping, max_length

In [ ]:
def window_sliding(iterable, n):
    gens = (
        it.chain(it.repeat(None, n - 1 - i), iterable, it.repeat(None, i))
        for i, gen in enumerate(it.tee(iterable, n)))
    return list(zip(*gens))

def find_treatments(tokens):
    found_treatments = []
    for x in range(max_treatment_length, 0, -1):
        for window in window_sliding(tokens, x):
            if tuple(window) in treatment_set:
                found_treatments.append(treatment_mapping[tuple(window)])
    return list(set(found_treatments))

In [ ]:
treatment_set, treatment_mapping, max_treatment_length = parse_treatment_definitons(open("data/treatment_definitons.txt", 'r'))

In [ ]:
def get_mapped_treatments(df):
    treatments = None
    merged_sentence = []
    for sent in (df[0], df[1], df[2]):
        if not sent:
            break
        tokens = tokenise(normalise(sent))
        mapped_treatments = find_treatments(tokens)
        if not mapped_treatments:
            break
        if treatments is None:
            treatments = mapped_treatments
            merged_sentence.append(sent)
        else:
            if set(mapped_treatments) - set(treatments):
                break
            merged_sentence.append(sent)
    df["treatments"] = treatments
    df["merged_sentence"] = " ".join(merged_sentence)
    return df

In [ ]:
def filter_stopwords(tokens):
    return [t for t in tokens if len(t) > 2 and t not in stopwords]

In [ ]:
def calculate_embeddings(tokens):
    token_embdgs = [embeddings.get(t) for t in tokens]
    token_embdgs = [t for t in token_embdgs if t is not None]
    return numpy.mean(token_embdgs, axis=0)

In [ ]:
with open('data/tinnitustalk_posts_filtered.csv', 'r') as fh:
    df = pandas.read_csv(fh, parse_dates=["timestamp"])

In [ ]:
df = df.loc[df['text'].notnull()]

In [ ]:
len(df.author_id.unique())

In [ ]:
df["text"] = df["text"].apply(lambda x: x.replace("\n", " "))

In [ ]:
df["sentences"] = df["text"].apply(lambda x: window_sliding(sentence_splitting(x), 3))

In [ ]:
n_df = pandas.concat([pandas.DataFrame(v, index=numpy.repeat(k,len(v))) for k,v in df.sentences.to_dict().items()])

In [ ]:
merged_df = pandas.concat([df, n_df], axis=1)

In [ ]:
df = merged_df.drop(["sentences", "position_in_thread", "thread_name", "text"], axis=1)

In [ ]:
df = df.apply(get_mapped_treatments, axis=1)

In [ ]:
df = df.loc[df['treatments'].notnull()]

In [ ]:
len(df.author_id.unique())

In [ ]:
df = df.drop([0, 1, 2], axis=1)

In [ ]:
df["tokens"] = df["merged_sentence"].apply(lambda x: tokenise(normalise(x)))

In [ ]:
df['filtered_tokens'] = df['tokens'].apply(filter_stopwords)

In [ ]:
word_counts = Counter()
for elem in df["filtered_tokens"]:
    word_counts += Counter(elem)

In [ ]:
word_counts.most_common(25)

In [ ]:
len(df.user_name.unique())

In [ ]:
df.to_csv("tt_with_treatments.csv")

In [ ]:
treatment_counts = Counter()
for elem in df["treatments"]:
    treatment_counts += Counter(elem)

In [ ]:
treatment_counts.most_common()